**IMPORT**

In [2]:
import os
import pandas as pd
import tensorflow as tf

In [3]:
def normalize_series(data, min, max):
    data-=min
    data/=max
    return data

def windowed_dataset(series, batch_size, n_past=24, n_future=4, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past+n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past+n_future))
    ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-n_future], w[-n_future:, :1]))
    ds = ds.batch(batch_size).prefetch(shift)
    return ds

In [6]:
# Retrieve dataset
df = pd.read_csv(os.path.join('lettuce_growth_days', 'lettuce_dataset.csv'), sep=',', encoding='ISO-8859-1')

# Delete column date, plant_id
df = df.drop(columns=['Plant_ID', 'Date'])

# Number of features in dataset
N_FEATURES = df.shape[1]

print(df.head())
print(F'n features: {N_FEATURES}')

# Normalize data
data = df.values
split_time = int(len(data)*0.8)
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

x_train = data[:split_time]
print(f'Train: {len(x_train)}')
x_valid = data[split_time:]
print(f'Test: {len(x_valid)}')

# DO NOT CHANGE THIS
BATCH_SIZE = 32
N_PAST = 24  # Number of past time steps based on which future observations should be predicted
N_FUTURE = 24  # Number of future time steps which are to be predicted.
SHIFT = 1  # By how many positions the window slides to create a new window of observations.

# Code to create windowed train and validation datasets.
# Complete the code in windowed_dataset.
# YOUR CODE HERE
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)
# YOUR CODE HERE
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)

model = tf.keras.models.Sequential([
    # tf.keras.layers.LSTM(64, 'relu', return_sequences=True, input_shape=[N_PAST, N_FEATURES]),
    tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(N_FUTURE, 'linear'),
])


class StopWhenReachDesireMAE(tf.keras.callbacks.Callback):
    def __init__(self, monitor='mae', monitor2='val_mae', target=0.14):
        super(StopWhenReachDesireMAE, self).__init__()
        self.monitor = monitor
        self.monitor2 = monitor2
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        current2 = logs.get(self.monitor2)
        if current is not None and current2 is not None:
            if current < self.target and current2 < self.target:
                print(
                    f'\nEpoch {epoch + 1}: {self.monitor} and {self.monitor2} have reached {self.target}. Stopping training.')
                self.model.stop_training = True

stop_callback = StopWhenReachDesireMAE('mae', 'val_mae', 0.05)

# Code to train and compile the model
# YOUR CODE HERE
model.compile(
    loss='mae',
    optimizer='adam',
    metrics=['mae']
)

model.fit(
    train_set,
    epochs=60,
    validation_data=valid_set,
    callbacks=[stop_callback]
)

# # Evaluate the model
# loss, mae = model.evaluate(X_test, y_test)
# print(f'Mean Absolute Error on test data: {mae}')
# 
# # Make predictions
# y_pred = model.predict(X_test)
# 
# # Optional: Convert predictions and actual values to a more readable format
# results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
# print(results.head())

   Temperature (°C)  Humidity (%)  TDS Value (ppm)  pH Level  Growth Days
0              33.4            53              582       6.4            1
1              33.5            53              451       6.1            2
2              33.4            59              678       6.4            3
3              33.4            68              420       6.4            4
4              33.4            74              637       6.5            5
n features: 5
Train: 2535
Test: 634


D:\Studi\MBKM\capstone\Capstone_Model_NPK\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.2316 - mae: 0.2316 - val_loss: 0.1523 - val_mae: 0.1523
Epoch 2/60


C:\Users\IDe\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0587 - mae: 0.0587 - val_loss: 0.1428 - val_mae: 0.1428
Epoch 3/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0566 - mae: 0.0566 - val_loss: 0.1515 - val_mae: 0.1515
Epoch 4/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0556 - mae: 0.0556 - val_loss: 0.1578 - val_mae: 0.1578
Epoch 5/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0548 - mae: 0.0548 - val_loss: 0.1595 - val_mae: 0.1595
Epoch 6/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0553 - mae: 0.0553 - val_loss: 0.1713 - val_mae: 0.1713
Epoch 7/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0545 - mae: 0.0545 - val_loss: 0.1786 - val_mae: 0.1786
Epoch 8/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0537 - mae: 0.0537 - val_loss: 0.1869 - val_mae: 0.1869
Epoch 9/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0528 - mae: 0.0528 - val_loss: 0.1886 - val_mae: 0.1886
Epoch 10/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0531 - mae: 0.0531 - v

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Load the data
data = pd.read_csv(os.path.join('lettuce_growth_days', 'lettuce_dataset.csv'), sep=',', encoding='ISO-8859-1')

# Convert 'Date' to numerical values if necessary
data['Date'] = pd.to_datetime(data['Date']).map(pd.Timestamp.toordinal)

# Normalize the input features
features = ['Date', 'Temperature (°C)', 'TDS Value (ppm)', 'pH Level']
target = 'Growth Days'

X = data[features]
y = data[target]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, monitor='mae', monitor1='val_mae', target=0.1):
        super(CustomCallback, self).__init__()
        self.monitor = monitor
        self.monitor1 = monitor1
        self.target = target
        
    def on_epoch_end(self, epoch, logs=None):
        if logs[self.monitor] is not None and logs[self.monitor1] is not None:
            if logs[self.monitor] < self.target and logs[self.monitor1] < self.target:
                print(f'\n{self.monitor}: {logs[self.monitor]} < {self.target}')
                print(f'\n{self.monitor1}: {logs[self.monitor1]} < {self.target}')
                self.model.stop_training = True

# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Linear activation for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

cust_callbacks = CustomCallback(monitor='mae', target=0.045)
# Train the model
history = model.fit(X_train, y_train, epochs=1000, validation_split=0.2, batch_size=32, callbacks=[cust_callbacks])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae}')

# Make predictions
y_pred = model.predict(X_test)

# Optional: Convert predictions and actual values to a more readable format
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
print(results.head())

Epoch 1/1000


D:\Studi\MBKM\capstone\Capstone_Model_NPK\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 717.4463 - mae: 23.2058 - val_loss: 545.2272 - val_mae: 19.7439
Epoch 2/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 506.2810 - mae: 18.9509 - val_loss: 210.5482 - val_mae: 12.1643
Epoch 3/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 151.9461 - mae: 10.2198 - val_loss: 35.2757 - val_mae: 4.7035
Epoch 4/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 29.1494 - mae: 4.2540 - val_loss: 16.9137 - val_mae: 3.0870
Epoch 5/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15.0290 - mae: 2.9842 - val_loss: 12.4073 - val_mae: 2.6944
Epoch 6/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step - loss: 11.0612 - mae: 2.5992 - val_loss: 10.6920 - val_mae: 2.5165
Epoch 7/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 966us/step - loss: 10.2729 - mae: 2.5653 - val_loss: 9.5055 - val_mae: 2.3780
Epoch 8/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 904us/step - loss: 8.6268 - mae: 2.3281 - val_loss: 8.2218 - val_mae: 2.2023
Epoch 9/1000
64/64 ━━━━━━━━━━━━━━━━━━━━ 

In [31]:
# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on test data: {mae}')

# Make predictions
y_pred = model.predict(X_test)

# Optional: Convert predictions and actual values to a more readable format
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
print(results.head())

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step - loss: 0.0029 - mae: 0.0358
Mean Absolute Error on test data: 0.034346576780080795
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step
      Actual  Predicted
254       25  25.022667
3162      40  39.994534
969       14  14.038104
940       30  29.991659
331       10   9.888810
